# Pokročilé dolování z databází - Průběžná zpráva
### Dan Balarin, Lukáš Vala
### Datasety: <a href = "https://archive-beta.ics.uci.edu/dataset/2/adult">Adult</a>, <a href="https://data.worldbank.org/indicator/NY.GDP.PCAP.CD?end=1994&most_recent_year_desc=true&start=1993">GDP in 1994</a>

# Vypracování návodu pro zpracování a analýzu dat pomocí systému CleverMiner
## Krok 0 Instalace potřebných knihoven
- Clever Miner: `pip install cleverminer`
- pandas: `pip install pandas`
## Krok 1 Načtení a zpracování dat
Prvním krokem je načíst data pomocí knihovny pandas. Data se v tomto případě nacházejí ve složce data:

In [ ]:
import pandas as pd

adult = pd.read_csv ('./data/adult.data', encoding='cp1250', sep=', ')
gdp = pd.read_csv ('./data/gdpStates.txt', encoding='cp1250', sep='\t')

print(adult.head(5))
print(gdp.head(5))

Pomocí parametru `sep` je důležité specifikovat, jaký separátor jednotlivých atributů je v tabulce použit. U datasetu adult je to `, ` a u GDP je to tabulátor. Správnost načtení dat můžeme poté zkontrolovat pomocí výpisu prvních 5 hodnot obou tabulek.

Dále bychom si také měli zkontrolovat, zda pandas správně určila datové typy u jednotlivých sloupců. 
To můžeme zjistit pomocí příkazu `dtypes`:

In [4]:
print(adult.dtypes)
print(gdp.dtypes)

Age                int64
Workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
country           object
income            object
dtype: object
country              object
Most Recent Year      int64
GDP per Capita      float64
dtype: object


Pro string vypíše pandas `object`, pro integer `int64` a pro desetinné číslo `float64`

## Spojení dat
Nyní je třeba spojit 2 tabulky do sebe. Obě obsahují sloupec country. V tabulce `adult` se tento sloupec jmenuje `native-country`, je proto nutné ho přejmenovat přímo v datasetu na `country`, aby se shodoval s názvem sloupce ve druhém datasetu. Dále je potřeba upravit jednotlivé hodnoty v jedné z tabulek, protože např. hodnota `United States` v tabulce adult je původně zapsána v gdp tabulce jako `United-States`. Použijeme tedy funkci definovanou v ukázce pod tímto textem a následně v datech GDP najdeme jednotlivé hodnoty a nahradíme je správnými hodnotami z tabulky adult (použita funkce najít a nahradit ve VS-code :) ).

In [5]:
def getUniqueValuesFromColumn(df, col):
    values = df[col].tolist()
    uniqueValues = []
    for i in values:
        if(i not in uniqueValues):
            uniqueValues.append(i)
    return uniqueValues

uniqueCountryValues = getUniqueValuesFromColumn(adult, "country")

Před samotným spojením tabulek ještě odstraníme z GDP tabulky ty státy, které nejsou obsaženy v tabulce adult, aby po spojení nedošlo k vytvoření několika řádků s prázdnými hodnotami (vyplněný by byl pouze stát). Použijeme proto další funkci, do které přidáme dataset, název sloupce a pole hodnot států. Funkce nám pak vráti dataset obsahující pouze řádky se státem, který byl obsažený v poskytnutém poli (uniqueCountryValues z části před touto).

In [6]:
def filter_rows_by_values(df, col, values):
    return df[df[col].isin(values)]

gdp = filter_rows_by_values(gdp, "country", uniqueCountryValues)

Poté již spojíme datasety a vznikne nám nový, který obsahuje všechny atributy z datasetu adult a navíc hrubý domácí produkt státu, ze kterého daná osoba pochází:

In [7]:
adultWithGdp = pd.merge(adult, gdp, on='country', how='outer')

Nyní můžeme vytvořit nové atributy v datasetu podle našich potřeb. Následující kód vytvoří nový sloupec `AgeStatus` a připíše hodnotu "Young" lidem mladším než 30, "Middle aged" lidem mezi 30 a 60 a "Old" lidem starším než 60.

In [8]:
adultWithGdp['AgeStatus'] = adultWithGdp.apply(lambda row: "Young" if row.Age < 30 else ("Middle aged" if row.Age >= 30 & row.Age < 60 else "Old"), axis=1)